Here, we present an implementation of a quantum algorithm for the discrete log problem on the multiplicative group. The algorithm is based on the explanation in the book **An Introduction to Quantum Computing**  by **P. Kaye, R. Laflamme, M. Mosca**. ([Link](http://mmrc.amss.cas.cn/tlb/201702/W020170224608149125645.pdf)). Refer section 7.4 of the book for details.

Here is the statement of discrete log problem for a group $G$.
Let $a, b \in G$. Let $b = a^k$. Our goal is to find $k$.

There are two main parts of the algorithm for $G=\mathbb{Z}^*_N$ where $N \in \mathbb{Z}^{\geq 2}$:

The first part is the implementation of the unitary transformation for modular exponentiation. This implementation is carried out in stages using the following functions: $\texttt{get-angles()}$, $\texttt{phi-add()}$, $\texttt{phi-add-mod-N()}$, $\texttt{multiply-mod-N()}$ and $\texttt{power-mod-N()}$. 

The implementation of the first four functions is based on the [this](https://arxiv.org/abs/quant-ph/0205095) paper by Stephane Beauregard and is heavily inspired by [this](https://qiskit.org/documentation/_modules/qiskit/aqua/algorithms/single_sample/shor/shor.html) Qiskit implementation of Shor's algorithm.

The second part is the reduction of the general discrete log problem to a set of discrete log problems where the order of generator is prime for each such new problem.

The basic reduction is explained in detail in Theorem A.2.1 in the book. Applying this reduction multiple times, we get the following result:

Let $|a| = r$ where $r \in \mathbb{Z}^{\geq 2}$. 
Also, let $r = p_1 \ldots p_n$ where $p_i$ is prime for $i=1,\ldots,n$,
let $r_i = p_i \ldots p_n$ for all $i \in \mathbb{Z}^+$,
let $l_i = \frac{r}{r_i}$ for all $i \in \mathbb{Z}^+$,
let $a_i = a^{\frac{r}{p_i}}$ for $i=1,\ldots,n$.
So, $|a_i| = p_i$ for $i=1,\ldots,n$.

Then $k$ can be written uniquely as
$$k = c_1r_2+\ldots+c_{n-1}r_n+c_n$$
where $c_i \in \mathbb{Z}_{p_i}$ for $i=1,\ldots,n$ and 
$b_i = a_i^{c_i}$ for $i=1,\ldots,n$ where
$b_i=(b a^{-(c_{i+1} r_{i+2} + \ldots c_n)})^{l_i}$ for $i=1,\ldots,n$.

So, solving discrete log problem for $(a_i, b_i, N)$ we can find $c_i$ for $i=1,\ldots,N$ using which we can find $k$.

In [1]:
import numpy as np
import math
from time import time
from math import gcd
from qiskit import(QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer)
from qiskit.aqua.circuits import FourierTransformCircuits as ftc
from qiskit.aqua.circuits.gates import mcu1
from qiskit.visualization import plot_histogram
from qiskit.quantum_info.operators import Operator
from qiskit.aqua.utils import summarize_circuits

$\texttt{mod-exp(a, b, N)}$ computes $a^b\text{ mod }N$ where $a \in \mathbb{Z}$, $b \in \mathbb{Z}^{\geq0}$ and $N \in \mathbb{Z}^+$.	

In [2]:
def mod_exp(a, b, N):
    X = int(a)
    E = int(b)
    m = int(N)
    if(E<0 or m<1):
        return None
    X = X % m
    Y = 1
    while E > 0:
        if E % 2 == 0:
            X = (X * X) % m
            E = E/2
        else:
            Y = (X * Y) % m
            E = E - 1
    return Y

$\texttt{egcd(a, b)}$, where  $a, b \in \mathbb{Z}$,  returns a 3-tuple $(g, x, y)$ where  $a, b, g \in \mathbb{Z}$ such that $|g| = \text{gcd}(a, b)$ and $ax + by = g$. If $a=b=0$, the $g=0$ is returned.

In [3]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

$\texttt{modinv(a, n)}$, where $a, n \in \mathbb{Z}$ such that $n \geq 2$, returns $a^{-1} \text{ mod } n$ if $a$ and $n$ are relatively prime and returns $\texttt{None}$ otherwise.

In [4]:
def modinv(a, n):
    if(n<2):
        return None
    g, _, x = egcd(n, a)
    if(g!=1):
        return None
    return x % n

$\texttt{prime-factorize(n)}$, where $n \in \mathbb{Z}^+$, returns two lists $p=(p_1, \ldots, p_m )$ and $a=(a_1, \ldots, a_m)$ such that $n = p_1^{a_1} \dots p_m^{a_m}$ where $p_i$ is prime and $a_i$ $\in \mathbb{Z}^+$ for $i=1,\ldots,m$ and $p_1 < \ldots < p_m$. If $n=1$, both lists are empty.

In [5]:
def prime_factorize(n):
    n = int(n)
    if n<1:
        return None
    if n<2:
        return [], []
    d=2
    pf = []
    power = []
    while d*d<=n:
        if n % d == 0:
            pf.append(d)
            k=0
            while n%d==0:
                n = n // d
                k = k + 1
            power.append(k)
        d = d + 1
    if n > 1:
        pf.append(n)
        power.append(1)
    
    return pf, power

$\texttt{prime-factorize-2(n)}$, where $n \in \mathbb{Z}^+$, returns a list $p = (p_1, \ldots, p_m)$ such that $n=p_1 \ldots p_m$ where $p_i$'s are prime and $p_1 \leq \ldots \leq p_m$. If $n=1$, $p$ is empty.

In [6]:
def prime_factorize_2(n):
    n = int(n)
    if(n<1):
        return None
    if n<2:
        return []
    d=2
    pf = []
    while d*d<=n:
        while n%d==0:
            n = n // d
            pf.append(d)
        d = d + 1
    if n > 1:
        pf.append(n)
    
    return pf

$\texttt{phi-n(n)}$, where $n \in \mathbb{Z}^+$, returns the value of Euler's totient function $\phi(n)$, which is the number of positive integers less than or equal to $n$ that are relatively prime to $n$.

In [7]:
def phi_n(n):
    n = int(n)
    if(n<1):
        return None
    pf , power = prime_factorize(n)
    v = 1
    for i in range(len(pf)):
        v *= (pf[i] ** (power[i] - 1 )) * (pf[i] - 1)
    return v

$\texttt{order(a, N)}$, where $a \in \mathbb{Z}$ and $N \in \mathbb{Z}^{\geq 2}$ such that $a$ and $N$ are relatively prime, returns the smallest positive integer $r$ such that $a^r \equiv 1 \text{ mod } N$.  One may replace it with Shor's order finding algorithm.

In [8]:
def order(a, N):
    N = int(N)
    a = int(a) % N
    if(N<2 or gcd(a, N)!=1):
        return None
    phi = phi_n(N)
    d = 1
    r = phi
    while d*d <= phi:
        if(phi % d == 0):
            if(mod_exp(a, d, N)==1):
                return d
            if(mod_exp(a, phi/d, N) == 1):
                r = phi/d
        d = d + 1
    return r

$\texttt{get-angles(a, n)}$ computes the rotation angle to be used in Fourier transform-like circuit for addition with $a$. $n$ denotes the number of bits used for representation. $a$ must be known classically.

In [9]:
def get_angles(a, n):
    bit = bin(abs(int(a)))[2:].zfill(n)
    theta = np.zeros(n)
    for i in range(n):
        for j in range(n-1-i, n):
            if bit[j] == '1':
                theta[i] += 2**(n-i-j-1)
    theta = theta * np.pi
    return theta

$\texttt{phi-add(circuit, q, a, n, c=[], inverse = False)}$ creates circuit to add(subtract, if $\texttt{inverse==True}$) $a$ to(from) $q$ modulo $2^n$ in Fourier Space. $n$ is the number of bits used for representation. $c$ is a (possibly empty) list of control bits. $a$ must be known classically. $q$ is a quantum register of $n$ bits.

In [10]:
def phi_add(circuit, q, a, n, c=[], inverse = False):
    theta = get_angles(a, n)    
    if(inverse):
        theta = -theta
    if not c:
        for i in range(n):
            circuit.u1(theta[i], q[i])    
    else:
        for i in range(n):
            circuit.mcu1(theta[i], c, q[i])    

$\texttt{phi-add-mod-N(circuit, q, a, n, N, c, aux, inverse = False)}$ creates circuit to add(subtract, if $\texttt{inverse==True}$) $a \in \mathbb{Z}^+$ to(from) $q$ mod $N$ in Fourier Space. $n$ is the number of bits used for representation. $c$ is a non-empty list of control bits. $a$ must be known classically. $q$ is a quantum register of $n+1$ bits. $q$ must be a non-negative integer less than $N$. $N$ must be a positive integer less than $2^n$. $aux$ is a single auxiliary qubit.

In [11]:
def phi_add_mod_N(circuit, q, a, n, N, c, aux, inverse = False):
    if not c :
        print('There should be at least one control qubit')
        return None
    if N <= 0 or N >= 2**n :
        print('Choose appropriate N')
        return None
    if(inverse):
        a = (-a) % N
    else: 
        a = a % N
        
    phi_add(circuit, q, a, n+1, c)
    phi_add(circuit, q, N, n+1, inverse=True)
    ftc.construct_circuit(circuit=circuit, qubits=[q[i] for i in reversed(range(n+1))], do_swaps=False, inverse=True)
    circuit.cx(q[n], aux)
    ftc.construct_circuit(circuit=circuit, qubits=[q[i] for i in reversed(range(n+1))], do_swaps=False)
    phi_add(circuit, q, N, n+1, [aux])
    phi_add(circuit, q, a, n+1, c, inverse=True)
    ftc.construct_circuit(circuit=circuit, qubits=[q[i] for i in reversed(range(n+1))], do_swaps=False, inverse=True)
    circuit.x(q[n])
    circuit.cx(q[n], aux)
    circuit.x(q[n])
    ftc.construct_circuit(circuit=circuit, qubits=[q[i] for i in reversed(range(n+1))], do_swaps=False)
    phi_add(circuit, q, a, n+1, c)

$\texttt{multiply-mod-N(circuit, q, a, n, N, c, aux)}$ creates circuit to multiply $a$ to $q$ mod $N$ in usual bit representation. $n$ is the number of bits used for representation. $a \in \mathbb{Z}^+$ must be known classically. $q$ is a quantum register of $n$ bits. $q$ must be a non-negative integer less than $N$ if we want the output to be $q$ mod $N$ (instead of $q$) when $c$ is $0$. $N$ must be a positive integer less than $2^n$. $a$ and $N$ must be relatively prime. $c$ is the only control bit. $aux$ is an auxiliary quantum register of $n+2$ bits.

In [12]:
def multiply_mod_N(circuit, q, a, n, N, c, aux):
    if N <= 0 or N >= 2**n :
        print('Choose appropriate N')
        return None
    a = a % N
    a_inv = modinv(a, N)
    if(not a_inv):
        print('Inverse of a does not exist')
        return None
    pow_2_mod = np.zeros(n)
    pow_2_mod[0] = 1
    for i in range(1, n):
        pow_2_mod[i] = (2*pow_2_mod[i-1]) % N
        
    ftc.construct_circuit(circuit=circuit, qubits=[aux[i] for i in reversed(range(n+1))], do_swaps=False)
    for i in range(n):
        phi_add_mod_N(circuit, aux, (pow_2_mod[i]*a) % N, n, N, [c, q[i]], aux[n+1])
    ftc.construct_circuit(circuit=circuit, qubits=[aux[i] for i in reversed(range(n+1))], do_swaps=False, inverse=True)
    
    for i in range(n):
        circuit.cswap(c, q[i], aux[i])

    ftc.construct_circuit(circuit=circuit, qubits=[aux[i] for i in reversed(range(n+1))], do_swaps=False)
    for i in reversed(range(n)):
        phi_add_mod_N(circuit, aux, (pow_2_mod[i]*a_inv) % N, n, N, [c, q[i]], aux[n+1], inverse=True)
    ftc.construct_circuit(circuit=circuit, qubits=[aux[i] for i in reversed(range(n+1))], do_swaps=False, inverse=True)

$\texttt{power-mod-N(circuit, q, r, a, m, n, N, aux)}$ creates circuit to compute $(r \cdot a^q )\text{ mod } N$ in the usual bit representation. $n$ is the number of bits used in the representation. $a$ must be known classically. $q$ and $r$ are quantum registers of $m$ and $n$ bits respectively. $N$ must be a positive integer less than $2^n$. $a$ and $N$ must be relatively prime. $aux$ is an auxiliary quantum register of $n+2$ bits. $r$ must be a non-negative integer less than $N$. However, if we know that $q$ is always positive, $r$ can be any non-negative integer less that $2^n$.

In [13]:
def power_mod_N(circuit, q, r, a, m, n, N, aux):
    if N <= 0 or N >= 2**n :
        print('Choose appropriate N')
        return None
    temp = a % N
    for i in range(m):
        multiply_mod_N(circuit, r, temp, n, N, q[i], aux)
        temp = (temp*temp) % N

$\texttt{discrete-log-solver(a, b, N, r, n-shots=64)}$ returns a pair $(t, n\_trials)$ such that $a^t \equiv b \text{ mod } N$ if such a $t$ exists. In this case, $t \in \mathbb{Z}_r$. Otherwise $t = \texttt{None}$. $0 \leq n\_trials \leq n\_shots$ is the number of times the algorithm was run to obtain the answer.  $a \in \mathbb{Z}$ and $N \in \mathbb{Z}^{\geq 2}$ such that  $a$ and $N$ must be relatively prime.  $b \in \mathbb{Z}$. $r$ is the order of $a$ modulo $N$. $r$ must be a prime number.  The function does not verify the primality of $r$. So, if $r$ is not prime, the behaviour is undefined. $n\_shots$ is the maximum number of times the algorithm should be run to determine the answer.

In [14]:
def discrete_log_solver(a, b, N, r, n_shots=64):
    N = int(N)
    a = int(a) % N
    b = int(b) % N
    r = int(r)
    n_shots = int(n_shots)

    if(N<2 or gcd(a, N)!=1 or n_shots<1):
        return None
    if(mod_exp(a, r, N)!=1):
        return None
    if(gcd(b, N)!=1):
        return (None, 0)
    
    n = int(np.ceil(np.log2(N)))
    m = int(np.ceil(np.log2(r))) + 1
    q = QuantumRegister(2*(m+n+1), 'q')
    c = ClassicalRegister(2*m, 'c')
    circuit = QuantumCircuit(q, c)
    
    for i in range(2*m):
        circuit.h(q[i])
    circuit.x(q[2*m])
    power_mod_N(circuit, q[:m], q[2*m:2*m+n], a, m, n, N, q[2*m+n:2*m+2*n+2])
    power_mod_N(circuit, q[m:2*m], q[2*m:2*m+n], b, m, n, N, q[2*m+n:2*m+2*n+2])
    ftc.construct_circuit(circuit=circuit, qubits=[q[i] for i in reversed(range(m))], inverse=True)
    ftc.construct_circuit(circuit=circuit, qubits=[q[i] for i in reversed(range(m, 2*m))], inverse=True)
    circuit.measure(q[:2*m], c[:2*m])
    
    simulator = Aer.get_backend('qasm_simulator')
    job = execute(circuit, simulator, shots=n_shots)
    result = job.result()
    counts = result.get_counts(circuit)

    i=1
    for key in counts:
        x = int(key[-m:], 2) 
        y = int(key[-2*m:-m], 2) 
        k = round((x*r)/(2**m))
        kt_mod_r = round((y*r)/(2**m))
        if(k % r != 0):
            t = (modinv(k, r) * kt_mod_r) % r
            if(mod_exp(a, t, N) == b):
                return (t, i)
                break
        i = i + 1
    return (None, n_shots)

$\texttt{discrete-log(a, b, N, n-shots=64)}$ returns a pair $(t, n\_trials)$ such that $a^t \equiv b \text{ mod } N$ if such a $t$ exists. In this case, $t \in \mathbb{Z}_r$. Otherwise $t = \texttt{None}$.  $a \in \mathbb{Z}$ and $N \in \mathbb{Z}^{\geq 2}$ such that  $a$ and $N$ must be relatively prime.  $b \in \mathbb{Z}$. Here, $r$ is the order of $a$ modulo $N$ which can also be composite. This function uses the $\texttt{order()}$ and $\texttt{prime-factorize-2()}$ functions. One may replace the definitions of these functions by their quantum counterparts which work in polynomial time. This is not done here so as to focus only on the quantum part of discrete log algorithm.  Also, the function works by repeatedly calling the $\texttt{discrete-log-solver()}$ function. The $n\_shots$ argument to this function will be the one passed in the $\texttt{discrete-log()}$  function.

In [26]:
def discrete_log(a, b, N, n_shots=64):
    N = int(N)
    a = int(a) % N
    b = int(b) % N
    n_shots = int(n_shots)
    
    if(N<2 or gcd(a, N) != 1 or n_shots<1):
        return None
    if(gcd(b, N) != 1):
        return (None, 0)
    
    r = int(order(a, N))
    if(r==1):
        if(b==1):
            return (0, 0)
        else:
            return (None, 0)
    pf = prime_factorize_2(r)
    n = len(pf)
    print('Output Format: ')
    print('Subproblem i: a_i b_i c_i time_taken_to_solve_the_subproblem')
    print('Subproblems are solved in reverse order \n')
    print('No. of subproblems:', n)
    A = np.zeros(n+1)
    R = np.zeros(n+2)
    L = np.zeros(n+1)
    L[1] = int(1)
    R[n+1] = int(1)
    k = 0
    n_trials = 0
    for i in reversed(range(1, n+1)):
        R[i] = R[i+1] * pf[i-1]
    for i in reversed(range(1, n+1)):
        print('Subproblem', i, ':', end=' ')
        L_i = r/R[i]
        A_i = mod_exp(a, r/pf[i-1], N)
        B_i = (b * mod_exp(a, (-k)%r, N) ) % N
        B_i = mod_exp(B_i, L_i, N)
        print(A_i, B_i, end=' ')
        s_time = time()
        C_i, trials = discrete_log_solver(A_i, B_i, N, pf[i-1], n_shots)
        print(C_i, round(time()-s_time), 's')
        n_trials = n_trials + trials
        if(C_i == None):
            return (None, n_trials)
        k = k + C_i * R[i+1]
    return (int(k) , n_trials)

In [29]:
a = 7
b = 19
N = 30
ans, n_trials = discrete_log(a, b, N)
print('\n')
print('ans:', ans)
print('n_trials:', n_trials)

Output Format: 
Subproblem i: a_i b_i c_i time_taken_to_solve_the_subproblem
Subproblems are solved in reverse order 

No. of subproblems: 2
Subproblem 2 : 19 1 0 37 s
Subproblem 1 : 19 19 1 37 s


ans: 2
n_trials: 8
